In [1]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
import csv
import dotenv
import os
from dotenv import load_dotenv, find_dotenv
from flipside import Flipside
import logging
# from degate.spot import Spot as Client
# from degate.lib.utils import config_logging
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
load_dotenv()
dotenv.load_dotenv('../../.env')
api_key = os.environ["DUNE_API_KEY"]
headers = {"X-Dune-API-Key": api_key}
flipside_key = os.environ["FLIPSIDE_API_KEY"]

In [3]:
# 28/02 last version to fetch trading data and pass it to clean_and_prepare_data function
import time
import logging
from ipynb.fs.full.dg5_get_close_time import get_latest_close_time, get_end_time_calculation
from ipynb.fs.full.dg4_clean_prepare_insert import clean_and_prepare_data
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
# Constants
BASE_URL_KLINES = "https://v1-mainnet-backend.degate.com/order-book-ws-api/klines"
GRANULARITY = 86400  
# START_TIME = 1699746810000 # contract creation time https://etherscan.io/tx/0x2306cd22386d3da63eb51edad872495f5f905996df5a0965c6d28cb66948d7aa 
# END_TIME = 1740139172000 # FEB 21, 11:59:59 PM   

# Load valid trading pairs
pairs_df = pd.read_csv("valid_trading_pairs.csv")

# Session for API reuse
session = requests.Session()

def fetch_trading_data_by_pair(pair, base_id, quote_id, start_time, end_time):
    """
    Fetch trading data for a given pair using session.
    """
    # pair, base_id, quote_id = pair_data["pair"], pair_data["base_id"], pair_data["quote_id"]
    
    params = {
        "base_token_id": base_id,
        "quote_token_id": quote_id,
        "granularity": GRANULARITY,
        "start": start_time, #START_TIME,
        "end": end_time #END_TIME
    }

    try:
        response = session.get(BASE_URL_KLINES, params=params, timeout=3)
        response.raise_for_status()
        result = response.json().get("data", [])

        if not result:
            return None

        df = pd.DataFrame(result)

        # Keep only expected columns (ignoring extra ones)
        # df = df[[col for col in df.columns if col in columns]]
        df.insert(0, "pair", pair) 
        # print(df)

        return df

    except requests.RequestException as e:
        print(f"Error fetching {pair}: {e}")
        return None

def fetch_all_trading_data(start_time=None, end_time=None, output_file=None):    
    all_data = []

    # allows multiple API requests to run concurrently, improving performance
    with ThreadPoolExecutor(max_workers=min(10, len(pairs_df))) as executor:
        
        # stores async results as a key and tradig pair as a value
        futures = { 
            # submits API call for each pair
            executor.submit(fetch_trading_data_by_pair, row["pair"], row["base_id"], row["quote_id"], start_time, end_time): row["pair"]
            for _, row in pairs_df.iterrows()
            }
        # collecting results form API calls
        for future in as_completed(futures):
            # gets the returned data and add it to all_data if not None
            if (result := future.result()) is not None:
                all_data.append(result)
   
   # Saving the data to a CSV file
    if all_data:
        # Generate a unique filename if none is provided
        if output_file is None:
            timestamp = time.strftime("%Y%m%d_%H%M%S")  # Example: 20250228_165430
            output_file = f"trading_data_{timestamp}.csv"

        pd.concat(all_data, ignore_index=True).to_csv(output_file, index=False) #test_raw_trading_data.csv
        print(f"INFO: Saved trading data to {output_file}.")

        # Pass the output file to the next function for further processing
        clean_and_prepare_data(output_file)
    
        return output_file
    
    else:
        logging.warning("No trading data retrieved.")
        return None





In [4]:
# First, fetch all trading data from the beginning of times
fetch_all_trading_data(1699746810000, 1740139172000)

2025-03-02 15:54:36,448 - INFO - File trading_data_20250302_155435.csv loaded successfully with shape (55972, 14)
2025-03-02 15:54:36,453 - INFO - Columns renamed successfully. New columns: ['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore', '_avg_price']
2025-03-02 15:54:36,458 - INFO - Columns dropped successfully. New columns: ['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades']


INFO: Saved trading data to trading_data_20250302_155435.csv.


2025-03-02 15:54:36,928 - INFO - Intermediate file saved as: None
2025-03-02 15:54:36,935 - INFO - Converted 'date' and 'close_time' to datetime format.
2025-03-02 15:54:37,823 - INFO - Data cleaned and prepared for upload to Dune.


'trading_data_20250302_155435.csv'

In [ ]:

fetch_all_trading_data((get_latest_close_time() + 1000), get_end_time_calculation())

2025-03-02 15:54:44,021 - INFO - Latest close_time found: 1740182399999
2025-03-02 15:54:50,959 - INFO - File trading_data_20250302_155450.csv loaded successfully with shape (1365, 14)
2025-03-02 15:54:50,960 - INFO - Columns renamed successfully. New columns: ['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades', 'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore', '_avg_price']
2025-03-02 15:54:50,961 - INFO - Columns dropped successfully. New columns: ['pair', 'date', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'trades']
2025-03-02 15:54:50,975 - INFO - Intermediate file saved as: None
2025-03-02 15:54:50,979 - INFO - Converted 'date' and 'close_time' to datetime format.
2025-03-02 15:54:51,006 - INFO - Data cleaned and prepared for upload to Dune.


INFO: Saved trading data to trading_data_20250302_155450.csv.


'trading_data_20250302_155450.csv'

In [7]:
import glob
file = sorted(glob.glob("trading_data_*.csv"))[-1]
file

'trading_data_20250302_155450.csv'

In [8]:
tst = pd.read_csv(sorted(glob.glob("trading_data_*.csv"))[-1])
tst.tail(1)

,pair,0,1,2,3,4,5,6,7,8,9,10,11,12
1364,RSR/USDC,1740787200000,0.008297,0.008297,0.008297,0.008297,0.0,1740873599999,0.0,0,0.0,0.0,True,0.0


NEXT STEP: automatization
- call token check functions
- call fetc_all_trading_data
